<a href="https://colab.research.google.com/github/changsin/Medium/blob/main/notebooks/JSON_Deserialization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to Deserialize from JSON
After serializing JSON objects, you need to deserialize them back to their original class objects. json.loads() is the main method for deserialization, just as json.dumps() is the main method for serialization.

json.loads() returns back the input string into a JSON dictionary so if you need to cast into the proper class type, you have to write some custimization methods.

In [102]:
import json

class Label:
    def __init__(self, label, x, y, width, height):
        self.label = label
        self.x = x
        self.y = y
        self.width = width
        self.height = height

    def __iter__(self):
        yield from {
            "label": self.label,
            "x": self.x,
            "y": self.y,
            "width": self.width,
            "height": self.height
        }.items()

    def __str__(self):
        return json.dumps(dict(self), ensure_ascii=False)

    def __repr__(self):
        return self.__str__()

    def to_json(self):
        return self.__str__()
    
    @staticmethod
    def from_json(json_dct):
      print(type(json_dct), json_dct['label'])
      return Label(json_dct['label'],
                   json_dct['x'], json_dct['y'],
                   json_dct['width'], json_dct['height'])

label = Label("person", 10, 10, 4, 10)
print(json.dumps(label.__dict__))

{"label": "person", "x": 10, "y": 10, "width": 4, "height": 10}


## 1. Use json.loads()
Load it as a JSON dictionary and convert it into the proper class object.

In [103]:
import json

json_str = "{\"label\": \"person\", \"x\": 10, \"y\": 10, \"width\": 4, \"height\": 10}"

json_dct = json.loads(json_str)

label_des = Label.from_json(json_dct)

print(label_des)

assert label_des.height == label.height

<class 'dict'> person
{"label": "person", "x": 10, "y": 10, "width": 4, "height": 10}


In [106]:
type(json_str)

str

## 2. object_hook
You can achieve the same thing by calling json.loads() with object_hook parameter. For simple classes, you can just pass the deserialization method directly.

In [104]:

label_decoded = json.loads(json_str, object_hook=Label.from_json)
print(type(label_decoded), type(label_des))
assert type(label_decoded) == type(label_des)

<class 'dict'> person
<class '__main__.Label'> <class '__main__.Label'>


In [86]:
json_str

'{"label": "person", "x": 10, "y": 10, "width": 4, "height": 10}'

## 3. Nested Classes
For nested classes, you have can do it in a couple of different ways. One is to do it on your own, parsing each dictionary entry and constructing the object dynamically. The other is to use object_hook and adding handling method for each nested class types.

In [87]:
json_str2 = "{ \
                \"version\": 1, \
                \"type\": \"bounding-box-labels\", \
                \"boundingBoxes\": { \
                \"20210715_111300 16.jpg\": [ \
                  { \
                    \"label\": \"StabilityOff\", \
                    \"x\": 1, \
                    \"y\": 1025, \
                    \"width\": 553, \
                    \"height\": 29 \
                  }, \
                  { \
                    \"label\": \"StabilityOn\", \
                    \"x\": 1, \
                    \"y\": 964, \
                    \"width\": 563, \
                    \"height\": 30 \
                  }\
                ] \
              }\
            }"

In [139]:
class ImageLabelCollection:
  def __init__(self, bboxes):
    self.version = 1
    self.type = "bounding-box-labels"
    self.bboxes = bboxes

  def __iter__(self):
      yield from {
          "version": self.version,
          "type": self.type,
          "boundingBoxes": self.bboxes
      }.items()

  def __str__(self):
      return json.dumps(self.to_json())

  def __repr__(self):
      return self.__str__()

  def to_json(self):
      to_return = {"version": self.version, "type": self.type}
      image_boxes = {}
      for key, boxes in self.bboxes.items():
          jboxes = []
          for box in boxes:
              jboxes.append(box.__dict__)
          image_boxes[key] = jboxes

      to_return["boundingBoxes"] = image_boxes
      return to_return
    
  @staticmethod
  def from_json(json_dct):
    if 'label' in json_dct.keys():
      return Label.from_json(json_dct)
    elif 'version' in json_dct.keys():
      return ImageLabelCollection(json_dct['boundingBoxes'])
    else:
      return json_dct

  @staticmethod
  def from_string(json_str):
    json_dct = json.loads(json_str)
    version = json_dct['version']
    obj_type = json_dct['type']
    bboxes_json = json_dct['boundingBoxes']
    bboxes = {}

    for image_name, bbs in bboxes_json.items():
      labels = []
      for bb in bbs:
        label_box = Label.from_json(bb)
        labels.append(label_box)

      bboxes[image_name] = labels

    return ImageLabelCollection(bboxes)

labeled_images = json.loads(json_str2, object_hook=ImageLabelCollection.from_json)

<class 'dict'> StabilityOff
<class 'dict'> StabilityOn


In [140]:
labeled_images

{"version": 1, "type": "bounding-box-labels", "boundingBoxes": {"20210715_111300 16.jpg": [{"label": "StabilityOff", "x": 1, "y": 1025, "width": 553, "height": 29}, {"label": "StabilityOn", "x": 1, "y": 964, "width": 563, "height": 30}]}}

In [141]:
ImageLabelCollection.from_string(json_str2)

<class 'dict'> StabilityOff
<class 'dict'> StabilityOn


{"version": 1, "type": "bounding-box-labels", "boundingBoxes": {"20210715_111300 16.jpg": [{"label": "StabilityOff", "x": 1, "y": 1025, "width": 553, "height": 29}, {"label": "StabilityOn", "x": 1, "y": 964, "width": 563, "height": 30}]}}

## 4. jsonpickle
A much easier solution for serialization and deserialization is using jsonpickle. It handles nested objects as well.

In [143]:
!pip install jsonpickle

In [148]:
import jsonpickle

labeled_images_pickled = jsonpickle.encode(labeled_images)

In [158]:
labeled_images_pickled

'{"py/object": "__main__.ImageLabelCollection", "version": 1, "type": "bounding-box-labels", "bboxes": {"20210715_111300 16.jpg": [{"py/object": "__main__.Label", "label": "StabilityOff", "x": 1, "y": 1025, "width": 553, "height": 29}, {"py/object": "__main__.Label", "label": "StabilityOn", "x": 1, "y": 964, "width": 563, "height": 30}]}}'

In [150]:
labeled_imaged_unpickled = jsonpickle.decode(labeled_images_pickled)

In [151]:
type(labeled_imaged_unpickled)

__main__.ImageLabelCollection

In [152]:
type(labeled_images_pickled)

str

In [153]:
labeled_imaged_unpickled.bboxes

{'20210715_111300 16.jpg': [{"label": "StabilityOff", "x": 1, "y": 1025, "width": 553, "height": 29},
  {"label": "StabilityOn", "x": 1, "y": 964, "width": 563, "height": 30}]}

In [157]:
type(labeled_imaged_unpickled.bboxes['20210715_111300 16.jpg'][0])

__main__.Label